# **Constraining the toy model using the RiboNpy workflow**

# Import related packages

In [ ]:
import cobra
import sys
import tempfile
import pandas as pd
import numpy as np
sys.path.append(r'./code/')
from cobra.io import load_json_model, write_sbml_model, validate_sbml_model
from pprint import pprint
from libsbml import *
from cobra import Model, Metabolite, Reaction
from ribonpy_core_functions import *

import json
import math
import re
import statistics
import os
import shutil
import random
from cobra.core import Reaction
from cobra.io.dict import model_to_dict
from cobra.util.solver import set_objective
from xml.dom import minidom
from optlang.symbolics import Zero, add
from cobra.flux_analysis import flux_variability_analysis

# 1 Construct Raw Ribozyme-Constrained Model

Input and output files

In [ ]:
#input files
# The genome-scale metabolic model for constructing the ribozyme-constrained model
model_name = './data/models/iML1515_new.xml'
obj_reaction = 'BIOMASS_Ec_iML1515_core_75p37M'

# Reaction-kcat file.
# eg. AADDGT,64.41327046 #s-1
reaction_kcat_file = './data/nc_machine_learning_kapp.json'
select_key = 'kappmax_ensemble_model_s-1'

# Gene-abundance file organized from PAXdb
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance-toy.csv"

# Gene-molecular_weight file organized from EcoCyc
# eg. b3500,48.77251
gene_molecular_weight_file = "./data/gene_molecular_weight-toy.csv"

reaction_gene_subunit_file = "./data/reaction_gene_subunit-toy.csv"
c13reaction_file = './data/C13reaction.csv' 

# Input ribozyme files
nucleotide_MW_file = "./data/nucleotide_MW.csv"
sequence_motifs_file = None
reactions_subset_file = None

In [ ]:
# Output files
gene_outfile = "./analysis/iML1515/genes.csv"
gpr_outfile = "./analysis/iML1515/all_reaction_GPR.csv"
reaction_gene_subunit_MW_file = "./analysis/iML1515/reaction_gene_subunit_MW.csv"
reaction_MW_file = "./analysis/iML1515/reaction_MW.csv"
reaction_kcat_MW_file = "./analysis/iML1515/modified_reaction_kcat_MW.csv"
json_output_file="./model/iML1515_irr_enz_ribozyme_constraint.json"
RiboNpy_fluxes_outfile = './analysis/iML1515/RiboNpy_ori_solution_df_pfba.csv'
GEM_fluxes_outfile = './analysis/iML1515/Orimodel_solution_df_pfba.csv'

# Output ribozyme files
ribozyme_data_file = './analysis/iML1515/ribozyme_data.csv'
modified_reaction_MW_file = './analysis/iML1515/modified_reaction_MW.csv'
modified_biosynthesis_reaction_MW_file = "./analysis/iML1515/modified_biosynthesis_reaction_MW.csv"
modified_reaction_kcat_MW_file = "./analysis/iML1515/modified_reaction_kcat_MW.csv"

Step1: preprocessing of the model

In [ ]:
model = cobra.io.read_sbml_model(model_name) 
convert_to_irreversible(model)
#split isoenzyme
model = isoenzyme_split(model)
model

Step2: Retrieving and calculating proteozyme and ribozyme kinetics and proteomics data

In [ ]:
[genes,gpr_relationship] = get_genes_and_gpr(model,gene_outfile,gpr_outfile)

In [ ]:
reaction_gene_subunit_MW = get_reaction_gene_subunit_MW(reaction_gene_subunit_file,gene_molecular_weight_file,reaction_gene_subunit_MW_file)
reaction_gene_subunit_MW.head(5)

In [ ]:
sequence_length = 79
nucleotide_proportion = [0.25,0.25,0.25,0.25]
new_ribozyme = True
random_proportion = False

ribozyme_MW = calculate_ribozyme_MW(nucleotide_MW_file, ribozyme_data_file, sequence_length, nucleotide_proportion, random_proportion, sequence_motifs_file, new_ribozyme)
ribozyme_MW.head(5)

In [ ]:
reaction_MW = calculate_reaction_mw(reaction_gene_subunit_MW_file,reaction_MW_file)
reaction_MW.head(5)

In [ ]:
new_reaction_selection = True
reaction_fraction = 1
selection_type = 'random'
specific_reactions = None

modified_reaction_MW, fraction_ribozyme_reactions = modify_reaction_MW(reaction_MW_file, modified_reaction_MW_file, reaction_fraction, ribozyme_data_file, selection_type, reactions_subset_file, specific_reactions, new_reaction_selection)
modified_reaction_MW.head(5)

In [ ]:
biosynth_costs_penalty = 1.320773
biosynth_costs = True

reaction_biosynthesis_costs = modify_reaction_biosynthesis_MW(modified_reaction_MW_file, modified_biosynthesis_reaction_MW_file, biosynth_costs_penalty, biosynth_costs)
reaction_biosynthesis_costs.head(5)

In [ ]:
ef_factor = 0.1

modify_reaction_kcat_mw = calculate_modified_reaction_kcat_mw(reaction_kcat_file, modified_biosynthesis_reaction_MW_file, modified_reaction_kcat_MW_file, select_key, ef_factor)
modify_reaction_kcat_mw.head(5)

In [ ]:
# calculate average MW, average kcat and average kcat_MW

# calculate average for each column
kcat_avg = modify_reaction_kcat_mw['kcat'].mean()
MW_avg = modify_reaction_kcat_mw['MW'].mean()
kcat_MW_avg = modify_reaction_kcat_mw['kcat_MW'].mean()

# print the results
print(f"Average kcat: {kcat_avg:.2f}")
print(f"Average MW: {MW_avg:.2f}")
print(f"Average kcat_MW: {kcat_MW_avg:.2f}")

# print(kcat_avg/MW_avg)

Step3: Save ribozyme concentration constraint model as json file.

In [ ]:
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)
print(upperbound)

In [ ]:
#create enzyme concentration constraint model
trans_model2enz_json_model_split_isoenzyme(model_name, modified_reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

enz_model=get_enzyme_constraint_model(json_output_file)

In [ ]:
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,RiboNpy_fluxes_outfile,reaction_kcat_MW_file)
print(enz_model_pfba_solution.fluxes[obj_reaction])


In [ ]:
norm_model = cobra.io.json.load_json_model(json_output_file)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv(GEM_fluxes_outfile)
print(norm_model_pfba_solution_df.fluxes[obj_reaction])

#Compare with C13 data
c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

# 2 Construct Final Ribozyme-Constrained Model

Input and output files

In [ ]:
#enyme model needed data
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

model_name = './data/models/iML1515_new.xml' 
c13reaction_file = './data/C13reaction.csv' 


In [ ]:
#max kcat for EC number selected from BRENDA and SABIO-RK database(use autoPACMEN)
kcat_database_combined_file= './data/Brenda_sabio_combined_select.json'

reaction_kcat_MW_file = './analysis/iML1515/modified_reaction_kcat_MW.csv' # for simplification, the reaction_kcat_MW_file name is used from here onwards, even if the path is the modified_reaction_kcat_MW.csv created in step 2
fluxes_infile_ori = './analysis/iML1515/RiboNpy_ori_solution_df_pfba.csv'
json_model_path = './model/iML1515_irr_enz_ribozyme_constraint.json'


In [ ]:
enz_ratio=0.01
reaction_enz_usage_file = './analysis/iML1515/RiboNpy_adj_round1_reaction_enz_usage_df.csv'
reaction_kcat_MW_round1_outfile = './analysis/iML1515/reaction_change_by_enzuse.csv'
json_round1_output_file = './model/iML1515_irr_enz_ribozyme_constraint_adj_round1.json'
round1_fluxes_outfile = './analysis/iML1515/RiboNpy_adj_round1_solution_df_pfba.csv'


In [ ]:
c13_percentage=0.1
json_round2_output_file= './model/iML1515_irr_enz_ribozyme_constraint_adj_round2.json'
reaction_kcat_MW_round2_outfile = './analysis/iML1515/reaction_change_by_c13.csv'
round2_fluxes_outfile = './analysis/iML1515/RiboNpy_adj_round2_solution_df_pfba.csv'

RiboNpy_solution_df_pfba_file='./analysis/iML1515/RiboNpy_solution_df_pfba.csv'

Step1: Calibration enzyme kcat according enzyme usage 

In [ ]:
enz_model=get_enz_model_use_enz_usage_by_eckcat(enz_ratio,json_model_path,fluxes_infile_ori,reaction_kcat_MW_file,\
                                      reaction_enz_usage_file,kcat_database_combined_file, model_name, \
                                      f, ptot, sigma, lowerbound, upperbound, json_round1_output_file, \
                                      reaction_kcat_MW_round1_outfile)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,round1_fluxes_outfile,reaction_kcat_MW_round1_outfile)
print(enz_model_pfba_solution.fluxes[obj_reaction])

c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

Step2: Calibration enzyme kcat according c13 reaction list

In [ ]:
enz_model=get_enz_model_use_c13(reaction_kcat_MW_round1_outfile, json_model_path, c13reaction_file, c13_percentage, \
                                kcat_database_combined_file,model_name, f, ptot, sigma, lowerbound, \
                                upperbound, json_round2_output_file,reaction_kcat_MW_round2_outfile)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,round2_fluxes_outfile,reaction_kcat_MW_round2_outfile)
print(enz_model_pfba_solution.fluxes[obj_reaction])

c13reaction_2_enz_model_diff = get_diff_reaction_use_c13(c13reaction_file,enz_model_pfba_solution)
print (c13reaction_2_enz_model_diff)

Step3: Solving ribozyme concentration constraint by COBRApy

In [ ]:
#run enzyme constraint metabolic model
enz_model=get_enzyme_constraint_model(json_round2_output_file)
#enz_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds = (13, 13)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
pfba_solution_df.to_csv(RiboNpy_solution_df_pfba_file)
print(pfba_solution.fluxes[obj_reaction])

In [ ]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_round2_output_file)
#norm_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds = (13, 13)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/iML1515/Orimodel_solution_df_pfba.csv')
print(pfba_solution.fluxes[obj_reaction])

# 4: all-in-one enzyme-constraint function

In [ ]:
model_name = './data/models/iML1515_new.xml' 
obj_reaction = 'BIOMASS_Ec_iML1515_core_75p37M'

sequence_length = 79
nucleotide_proportion = [0.25,0.25,0.25,0.25]
random_proportion = False
new_ribozyme = True

reaction_fraction = 0
selection_type = 'random'
new_reaction_selection = True

biosynth_costs = False
biosynth_costs_penalty = 1.320773

ef_factor = 0.1

obj_solution = get_ribozyme_constraint_model_pfba_obj_solution(model_name, obj_reaction, sequence_length, nucleotide_proportion, random_proportion, new_ribozyme, reaction_fraction, selection_type, new_reaction_selection, biosynth_costs, biosynth_costs_penalty, ef_factor)